In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# =======================
# Configuración API
# =======================
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Prompt 1 — Base (mínimo viable)

**Descripción técnica**  
Genera un itinerario simple para un destino y cantidad de días usando un único prompt natural. No hay estructura JSON ni validaciones: el modelo responde en texto libre.

**Tecnología aplicada**  
- Python + `dotenv` para credenciales.  
- `openai` Chat Completions con el modelo `gpt-4o-mini`.  

**Metodología**  
- *Zero-shot prompting*: instrucciones breves sin formato rígido.  
- Salida directa a consola.  

**Notas**  
Este es el punto de partida: sin manejo de errores, sin costos/tokenes, sin helpers, sin imágenes.


In [2]:
# =======================
# Datos precargados
# =======================
destino = "Mendoza"
cant_dias = 3

# =======================
# Prompt 1 — Básico
# =======================
prompt1 = f"""
Organizá un viaje de {cant_dias} días a {destino}.
Incluí actividades, comidas y presupuesto.
"""

response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt1}],
    temperature=0.7
)

print("=== Prompt 1 — Respuesta ===")
print(response1.choices[0].message.content)

=== Prompt 1 — Respuesta ===
¡Claro! Aquí tienes un itinerario para un viaje de 3 días a Mendoza, Argentina, que incluye actividades, comidas y un presupuesto estimado.

### Itinerario de 3 días en Mendoza

#### Día 1: Llegada a Mendoza y Exploración de la Ciudad

**Mañana:**
- **Llegada a Mendoza**: Toma un vuelo a Mendoza y trasládate a tu hotel. 
- **Alojamiento**: Hotel de 3 estrellas (por ejemplo, Hotel Internacional o similar). Costo aproximado: $80 - $120 por noche.

**Tarde:**
- **Almuerzo en el centro**: Prueba una parrillada en "El Patio de Jesús María". Costo aproximado: $15 - $25 por persona.
- **Recorrido por la ciudad**: Visita la Plaza Independencia, el Parque General San Martín y el Cerro de la Gloria. 
- **Copa de vino en una vinoteca local**: Visita "La Enoteca" para degustar vinos locales. Costo aproximado: $10 por persona.

**Noche:**
- **Cena en "Azafrán"**: Disfruta de una cena gourmet con maridaje de vinos. Costo aproximado: $30 - $50 por persona.

**Presupuesto 

In [3]:
# =======================
# Datos precargados
# =======================

destino = "Mendoza"
cant_dias = 3
cant_personas = 2
presupuesto = "200000"
moneda = "ARS"
intereses = "Deportes Invernales"
modo_viaje = "Exprímelo"

## Prompt 2 — Intermedio (más parámetros, salida guiada)

**Descripción técnica**  
Amplía el contexto con más parámetros (personas, presupuesto, moneda, intereses) y solicita un itinerario día por día con horarios aproximados y un presupuesto dividido por rubros.

**Tecnología aplicada**  
- `openai` (`gpt-4o-mini`) con *temperature* más baja para mejorar coherencia.  

**Metodología**  
- *Role assignment* Especificacion de Rol (Actuá como organizador de escapadas) 
- *Instruction prompting* con especificación de secciones (actividades, comidas, presupuesto por rubros).  

**Mejoras respecto a Prompt 1**  
- Más datos de entrada (personas/presupuesto/moneda/intereses).  
- Pedido más estructurado (día por día + desglose de costos).  
- Aún **no** hay JSON ni validación programática.


In [4]:
# =======================
# Prompt 2 — Intermedio
# =======================
prompt2 = f"""
Actuá como organizador de escapadas. 
Con estos datos: destino={destino}, días={cant_dias}, personas={cant_personas}, presupuesto={presupuesto} {moneda}, intereses={intereses}, modo={modo_viaje}.
Armá un itinerario día por día con horarios aproximados y comidas. 
Incluí un presupuesto estimado dividido en transporte, alojamiento y actividades.
"""

response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt2}],
    temperature=0.5
)

print("=== Prompt 2 — Respuesta ===")
print(response2.choices[0].message.content)

=== Prompt 2 — Respuesta ===
¡Claro! Aquí tienes un itinerario para una escapada de 3 días a Mendoza, enfocado en deportes invernales, para 2 personas, con un presupuesto de 200,000 ARS.

### Itinerario de Escapada a Mendoza

#### **Día 1: Llegada a Mendoza**

**08:00 - Salida desde tu ciudad**
- **Transporte:** Viaje en avión o colectivo a Mendoza.
- **Presupuesto estimado:** 40,000 ARS (ida y vuelta para 2 personas).

**10:30 - Llegada a Mendoza**
- **Transporte:** Traslado al alojamiento en taxi o transfer.
- **Presupuesto estimado:** 3,000 ARS.

**11:30 - Check-in en el hotel**
- **Alojamiento:** Hotel 3 estrellas en el centro de Mendoza.
- **Presupuesto estimado:** 30,000 ARS (2 noches).

**12:30 - Almuerzo**
- **Lugar:** Restaurante local.
- **Presupuesto estimado:** 4,000 ARS.

**14:00 - Actividad en la montaña**
- **Actividad:** Excursión a la estación de esquí (ej. Cerro Catedral o Los Penitentes).
- **Transporte:** Alquiler de auto o excursión organizada.
- **Presupuesto esti

## Prompt 3 — Avanzado (estructura JSON y parsing)

**Descripción técnica**  
Solicita **únicamente JSON** con campos: `parametros`, `supuestos`, `presupuesto` y `itinerario`. Se incluye el helper `safe_json_parse` para limpiar *code fences* y convertir la respuesta a diccionario Python.

**Tecnología aplicada**  
- `openai` (`gpt-4o-mini`) + módulo `json`.  

**Metodología**  
- *Structured output prompting*: el modelo debe responder en JSON válido.  
- Parsing y `print` *pretty* de la respuesta.  

**Mejoras respecto a Prompt 2**  
- Respuesta estructurada en JSON.  
- Helper para tolerar respuestas con ```json ...```.  
- Base para validaciones y posterior post-proceso.


In [5]:
import os, json

# =======================
# Helper JSON
# =======================
def safe_json_parse(raw_text: str):
    cleaned = raw_text.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("`")
        cleaned = cleaned.replace("json\n", "").replace("json", "", 1).strip()
    try:
        return json.loads(cleaned)
    except:
        return {}

# =======================
# Prompt 3 — Avanzado
# =======================
prompt3 = f"""
Actuá como organizador de escapadas de 2–3 días. 
Datos: destino={destino}, días={cant_dias}, personas={cant_personas}, presupuesto={presupuesto} {moneda}, intereses={intereses}, modo={modo_viaje}.
Si falta información, proponé supuestos y aclarálo en un campo "supuestos".

Devolvé SOLO un JSON con:
- parametros
- supuestos
- presupuesto general
- itinerario simple de {cant_dias} días (con actividades y comidas por día).
"""

response3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt3}],
    temperature=0.3
)

raw3 = response3.choices[0].message.content
itinerario_json3 = safe_json_parse(raw3)

print("=== Prompt 3 — Respuesta JSON ===")
print(json.dumps(itinerario_json3, indent=2, ensure_ascii=False))

=== Prompt 3 — Respuesta JSON ===
{
  "parametros": {
    "destino": "Mendoza",
    "días": 3,
    "personas": 2,
    "presupuesto": 200000,
    "intereses": "Deportes Invernales",
    "modo": "Exprímelo"
  },
  "supuestos": {
    "temporada": "invierno",
    "tipo de alojamiento": "hotel 3 estrellas",
    "transporte": "auto alquilado",
    "comidas": "comidas en restaurantes locales"
  },
  "presupuesto_general": {
    "alojamiento": 60000,
    "transporte": 30000,
    "comidas": 50000,
    "actividades": 80000,
    "total": 200000
  },
  "itinerario": {
    "día_1": {
      "actividades": [
        "Llegada a Mendoza y alquiler de auto",
        "Visita a la estación de esquí Los Penitentes",
        "Esquí o snowboard durante la tarde"
      ],
      "comidas": [
        "Almuerzo en el restaurante de la estación de esquí",
        "Cena en un restaurante local en Mendoza"
      ]
    },
    "día_2": {
      "actividades": [
        "Desayuno en el hotel",
        "Excursión a la m

In [6]:
from openai import OpenAI
import os, json
from google import genai
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO

# =======================
# Configuración API
# =======================
load_dotenv()
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# =======================
# Datos precargados
# =======================
destino = "Mendoza"
cant_dias = 3
cant_personas = 2
presupuesto = "200000"
moneda = "ARS"
intereses = "Deportes Invernales"
modo_viaje = "Exprímelo"
fecha_tentativa = "20/7/2025"

# =======================
# Helper JSON
# =======================
def safe_json_parse(raw_text: str):
    cleaned = raw_text.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("`")
        cleaned = cleaned.replace("json\n", "").replace("json", "", 1).strip()
    try:
        return json.loads(cleaned)
    except:
        return {}

## Prompt 4 — Optimizado (normalización + criterios + imágenes)

**Descripción técnica**  
Pide **normalizar parámetros**, explicitar **supuestos**, definir **criterios de planificación** y **alertas**, y devolver un **itinerario detallado por bloques horarios**. Se fuerza *JSON-only* vía `system`. Además, **genera imágenes** (Mapa, Flyer, Infografía) con **Gemini 2.5 Flash Image** y guarda los PNG.

**Tecnología aplicada**  
- `openai` (`gpt-4o-mini`) para el JSON.  
- `google.genai` (Gemini 2.5 Flash Image) + `Pillow` para PNG.  
- Cálculo y *print* de **tokens usados**.  

**Metodología**  
- Instrucciones estrictas + esquema JSON.  
- Post-proceso: extracción de puntos del itinerario para construir prompts de imagen.  

**Mejoras respecto a Prompt 3**  
- JSON más rico (criterios, alertas, bloques horarios).  
- Visibilidad de tokens/costos.  
- Generación de imágenes complementarias.


In [7]:
# =======================
# Prompt 4 — Optimizado (todo en una salida)
# =======================
prompt4 = f"""
Actuá como un organizador de escapadas de 2–3 días. 

Usá exclusivamente los siguientes datos del usuario:

- Destino: {destino}
- Días: {cant_dias}
- Personas: {cant_personas}
- Presupuesto: {presupuesto} {moneda}
- Intereses: {intereses}
- Modo de viaje: {modo_viaje}
- Fecha tentativa: {fecha_tentativa}

Normalizá esta información y devolvé SOLO un JSON con:

- parametros normalizados (destino, días, personas, presupuesto, intereses, modo, fechas).
- supuestos explícitos.
- criterios de planificación (densidad, descansos, gasto, franja horaria).
- alertas relevantes.
- itinerario DETALLADO de {cant_dias} días (bloques horarios: mañana, mediodía, tarde, noche, comidas, traslados, alternativas low-cost/premium).
"""

response4 = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Sos un planificador de viajes. Respondé SOLO con un JSON válido y completo."},
        {"role": "user", "content": prompt4}
    ],
    temperature=0.3
)
print("Tokens usados:", response4.usage.total_tokens)

raw4 = response4.choices[0].message.content
itinerario_json4 = safe_json_parse(raw4)
if not itinerario_json4:
    print("⚠️ Error: el modelo no devolvió un JSON válido.")

print("=== Prompt 4 — Respuesta JSON ===")
print(json.dumps(itinerario_json4, indent=2, ensure_ascii=False))

# =======================
# AGREGADO: Generación de imágenes con Gemini 2.5 Flash Image
# =======================
puntos = []
for dia, actividades in itinerario_json4.get("itinerario", {}).items():
    if isinstance(actividades, list):
        puntos.append(f"{dia}: " + ", ".join(actividades))
lista_puntos = " | ".join(puntos)

mapa_prompt = f"Mapa ilustrado minimalista de {destino} para {cant_dias} días. Puntos: {lista_puntos}. Paleta cálida, formato 16:9."
flyer_prompt = f"Flyer turístico 16:9 'Escapada a {destino}', subtítulo {intereses}, estilo travel poster moderno."
infografia_prompt = f"Infografía 16:9 para {destino}, mostrando % de gastos y % de tiempo. Íconos simples y etiquetas claras."

for nombre, p in [("Mapa", mapa_prompt), ("Flyer", flyer_prompt), ("Infografia", infografia_prompt)]:
    response_img = gemini_client.models.generate_content(
        model="gemini-2.5-flash-image-preview",
        contents=[p],
    )
    for part in response_img.candidates[0].content.parts:
        if part.inline_data:
            img = Image.open(BytesIO(part.inline_data.data))
            img.save(f"prompt4_{nombre}.png")
            print(f"{nombre} generado → prompt4_{nombre}.png")

Tokens usados: 1028
=== Prompt 4 — Respuesta JSON ===
{
  "parametros_normalizados": {
    "destino": "Mendoza",
    "dias": 3,
    "personas": 2,
    "presupuesto": 200000,
    "intereses": "Deportes Invernales",
    "modo": "Exprímelo",
    "fechas": "20/07/2025"
  },
  "supuestos_explicitos": {
    "temporada": "alta",
    "disponibilidad_de_actividades": true,
    "clima": "frío con posibilidad de nieve",
    "transporte": "auto alquilado"
  },
  "criterios_de_planificacion": {
    "densidad": "alta",
    "descansos": "mínimos",
    "gasto": "controlado, priorizando actividades",
    "franja_horaria": "completa"
  },
  "alertas_relevantes": [
    "Reservar actividades de esquí con anticipación",
    "Verificar condiciones climáticas antes de viajar",
    "Considerar la posibilidad de cambios en el itinerario por mal tiempo"
  ],
  "itinerario_detallado": {
    "dia_1": {
      "mañana": {
        "actividad": "Llegada a Mendoza y traslado a la estación de esquí",
        "comida": 

## Prompt 5 — Ultra (pipeline en 2 etapas + utilidades)

**Descripción técnica**  
Separa el flujo en **dos prompts**:  
1) *Intake* → JSON normalizado (param/supuestos/ajustes/criterios/presupuesto/checklist/alertas/QA).  
2) *Itinerario* → genera un plan detallado en **Markdown** usando el JSON del intake.  
Incluye utilidades: cálculo de **tokens por etapa**, guardado a archivo, *builder* de prompts visuales y **función** para generar imágenes con Gemini.

**Tecnología aplicada**  
- `openai` (`gpt-4o-mini`) para intake + itinerario.  
- `google.genai` (Gemini 2.5 Flash Image) + `Pillow`.  

**Metodología**  
- *Chain-of-prompts*: output estructurado → consumo por segundo prompt.  
- Reutilización: función de generación de imágenes para evitar código duplicado.  

**Mejoras respecto a Prompt 4**  
- Separación de responsabilidades (intake vs. rendering del itinerario).  
- Medición de costos por paso.  
- Código más modular y reutilizable.


In [8]:
# =======================
# Helper TOKENS
# =======================

def calcular_costo(response, nombre="Prompt"):
    if not hasattr(response, "usage"):
        return
    in_tokens = response.usage.prompt_tokens
    out_tokens = response.usage.completion_tokens
    total_tokens = response.usage.total_tokens
    in_price = in_tokens * 0.00000015
    out_price = out_tokens * 0.0000006
    print(f"*** {nombre} → Tokens usados: {total_tokens} (entrada={in_tokens}, salida={out_tokens}), USD {in_price+out_price:.6f}")

# =======================
# Prompt 5 — Intake ultra optimizado
# =======================
intake_prompt5 = f"""
Sos un organizador experto en escapadas de 2–3 días. 
Devolvé SIEMPRE un JSON válido.

IMPORTANTE:
- Usá EXCLUSIVAMENTE los parámetros precargados.
- No inventes ni cambies destino, fechas, moneda, intereses ni modo.
- Si falta info → en "supuestos". Si es inválida → en "ajustes".

Datos del usuario:
- Destino: {destino}
- Días: {cant_dias}
- Personas: {cant_personas}
- Presupuesto: {presupuesto} {moneda}
- Intereses: {intereses}
- Modo: {modo_viaje}
- Fecha: {fecha_tentativa}

Salida esperada:
{{
  "parametros": {{...}},
  "supuestos": [],
  "ajustes": [],
  "criterios": {{}},
  "presupuesto": {{}},
  "checklist": [],
  "alertas": [],
  "qa": []
}}
"""

intake_response5 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Respondé SOLO con JSON válido."},
        {"role": "user", "content": intake_prompt5}
    ],
    temperature=0.2
)

calcular_costo(intake_response5, "Prompt 5 - Intake")
raw_intake5 = intake_response5.choices[0].message.content
intake_json5 = safe_json_parse(raw_intake5)

print("=== Prompt 5 — Intake JSON ===")
print(json.dumps(intake_json5, indent=2, ensure_ascii=False))

# =======================
# Prompt 5 — Itinerario ultra optimizado
# =======================
itinerario_prompt5 = f"""
Usá este contexto JSON:

{json.dumps(intake_json5, indent=2, ensure_ascii=False)}

Generá un itinerario DETALLADO de {cant_dias} días en Markdown limpio.

Formato:
Día N — Zona / tema principal
09:00-11:00 Actividad (lugar)
11:15-13:00 Actividad (lugar)
13:15-14:30 Almuerzo (opciones)
15:00-17:00 Actividad (lugar)
17:15-19:00 Actividad (lugar)
20:00 Cena (opciones)

Incluir: traslados, resumen, tips, presupuesto bajo/medio/alto ({moneda}), alertas QA.
"""

itinerario_response5 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Devolvé solo itinerario en Markdown."},
        {"role": "user", "content": itinerario_prompt5}
    ],
    temperature=0.4
)

calcular_costo(itinerario_response5, "Prompt 5 - Itinerario")
content5 = itinerario_response5.choices[0].message.content

print("\n=== Itinerario Ultra generado ===")
print(content5)

with open("itinerario_ultra.txt", "w", encoding="utf-8") as f:
    f.write(content5)

# =======================
# Builder para Prompts de Imagen basados en el itinerario
# =======================
def build_image_prompts(destino, cant_dias, intereses, itinerario_text):
    puntos = []
    for line in itinerario_text.splitlines():
        if "Actividad" in line or "Almuerzo" in line or "Cena" in line:
            puntos.append(line.strip("- *"))

    lista_puntos = " | ".join(puntos)

    mapa_prompt = f"""
Mapa ilustrado minimalista de {destino} para una escapada de {cant_dias} días.
Marcar con íconos las paradas clave:
{lista_puntos}
Leyenda: Día 1 a Día {cant_dias}.
Líneas suaves de conexión, tipografías legibles, paleta cálida. Formato 16:9.
Devuelve únicamente la imagen del mapa en PNG, sin texto adicional.
"""

    flyer_prompt = f"""
Flyer turístico 16:9 para {destino}, estilo travel-poster moderno.
Título: 'Escapada a {destino}'.
Subtítulo: {intereses}.
Imagen icónica del lugar, composición limpia, margen seguro para texto.
Devuelve únicamente la imagen en PNG, sin texto adicional.
"""

    infografia_prompt = f"""
Infografía clara para {destino}, formato 16:9.
Incluye dos gráficos simples:
- Gráfico circular de gastos: 30% transporte, 25% alojamiento, 25% comidas, 20% actividades.
- Gráfico de barras de tiempo: 40% actividades, 30% traslados, 20% descanso, 10% comidas.
Iconos simples y etiquetas legibles.
Devuelve únicamente la imagen en PNG, sin texto adicional.
"""

    return [
        ("Mapa", mapa_prompt),
        ("Flyer", flyer_prompt),
        ("Infografia", infografia_prompt)
    ]

# =======================
# Función optimizada para generar imágenes (fix binarios)
# =======================
def generar_imagenes(prompts, prefix="output"):
    for nombre, p in prompts:
        try:
            print(f"\n=== Generando {nombre} ===")
            response_img = gemini_client.models.generate_content(
                model="gemini-2.5-flash-image-preview",
                contents=[p],
            )

            saved = False
            for part in response_img.candidates[0].content.parts:
                if hasattr(part, "inline_data") and part.inline_data:
                    img = Image.open(BytesIO(part.inline_data.data))
                    img.save(f"{prefix}_{nombre}.png")
                    print(f"✅ {nombre} generado → {prefix}_{nombre}.png")
                    saved = True

            if not saved:
                print(f"⚠️ No se generó imagen para {nombre}. Gemini no devolvió inline_data")

        except Exception as e:
            print(f"❌ Error generando {nombre}: {e}")

# =======================
# Uso con el itinerario generado
# =======================
prompts_img = build_image_prompts(destino, cant_dias, intereses, content5)
generar_imagenes(prompts_img, prefix="prompt5")

*** Prompt 5 - Intake → Tokens usados: 332 (entrada=214, salida=118), USD 0.000103
=== Prompt 5 — Intake JSON ===
{
  "parametros": {
    "destino": "Mendoza",
    "días": 3,
    "personas": 2,
    "presupuesto": 200000,
    "intereses": "Deportes Invernales",
    "modo": "Exprímelo",
    "fecha": "20/7/2025"
  },
  "supuestos": [],
  "ajustes": [],
  "criterios": {},
  "presupuesto": {},
  "checklist": [],
  "alertas": [],
  "qa": []
}
*** Prompt 5 - Itinerario → Tokens usados: 961 (entrada=269, salida=692), USD 0.000456

=== Itinerario Ultra generado ===
# Itinerario de 3 Días en Mendoza - Deportes Invernales

## Día 1 — Valle de Uco / Esquí
09:00-11:00 Esquí en Cerro Castor (Estación de Esquí)  
*Traslado: 1 hora desde Mendoza*

11:15-13:00 Clases de esquí para principiantes (Cerro Castor)  

13:15-14:30 Almuerzo  
*Opciones: Restaurante en la base de la montaña o picnic con productos locales*

15:00-17:00 Esquí libre (Cerro Castor)  

17:15-19:00 Visita a la bodega de la zona (Bode

## Prompt 5 Lite — Minimalista (tokens bajos + mapa/flyer)

**Descripción técnica**  
Versión liviana que mantiene intake en JSON **breve** y un itinerario **simple**. La generación visual se reduce a **Mapa** y **Flyer**. El **Mapa** se adapta dinámicamente al **itinerario** (extrae puntos clave del texto). **Sin infografía** para evitar textos de mala calidad.

**Tecnología aplicada**  
- `openai` (`gpt-4o-mini`) para intake + itinerario.  
- `google.genai` (Gemini 2.5 Flash Image) + `Pillow`.  
- *Prompts* visuales centralizados en un **diccionario** y función única para exportar PNG.  

**Metodología**  
- *Cost-aware prompting*: menos tokens y salidas más simples.  
- Dinamización del mapa a partir del itinerario generado.  

**Mejoras respecto a Prompt 5**  
- Menor consumo de tokens (prompts y salidas más cortos).  
- Eliminación de la infografía.  
- Mapa con **fondo de color** y **sin texto**, alimentado por el itinerario.


In [9]:
# =======================
# Prompt 5 Lite — Intake
# =======================
intake_prompt_lite = f"""
Sos un organizador de {cant_dias} días.
Devolvé SOLO un JSON breve.

IMPORTANTE:
- Usá EXCLUSIVAMENTE los parámetros precargados.
- No inventes ni cambies destino, fechas, moneda, intereses ni modo.

JSON esperado:
{{
  "param": {{
    "dest": "{destino}",
    "dias": {cant_dias},
    "pers": {cant_personas},
    "presup_pp": {{"m": {presupuesto}, "mon": "{moneda}"}},
    "int": ["{intereses}"],
    "modo": "{modo_viaje}",
    "fechas": "{fecha_tentativa}"
  }},
  "sup": [],
  "ajus": [],
  "crit": {{}},
  "presup": {{}},
  "chk": [],
  "alert": [],
  "qa": []
}}
"""

intake_response_lite = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Respondé SOLO con JSON válido y breve."},
        {"role": "user", "content": intake_prompt_lite}
    ],
    temperature=0.2
)

calcular_costo(intake_response_lite, "Prompt 5-Lite - Intake")
raw_intake_lite = intake_response_lite.choices[0].message.content
intake_json_lite = safe_json_parse(raw_intake_lite)

print("=== Prompt 5 Lite — Intake JSON ===")
print(json.dumps(intake_json_lite, indent=2, ensure_ascii=False))

# =======================
# Prompt 5 Lite — Itinerario
# =======================
itinerario_prompt_lite = f"""
Usá este JSON:

{json.dumps(intake_json_lite, indent=2, ensure_ascii=False)}

Generá un itinerario simple de {cant_dias} días.

Formato:
Día N - Zona
09:00-11:00 Actividad
11:15-13:00 Actividad
13:15-14:30 Almuerzo (2 opciones)
15:00-17:00 Actividad
17:15-19:00 Actividad
20:00 Cena (2 opciones)

Incluí traslados, resumen, tips, presupuesto ({moneda}), alertas QA.
"""

itinerario_response_lite = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Devolvé solo itinerario en texto limpio."},
        {"role": "user", "content": itinerario_prompt_lite}
    ],
    temperature=0.4
)

calcular_costo(itinerario_response_lite, "Prompt 5-Lite - Itinerario")
content_lite = itinerario_response_lite.choices[0].message.content

print("\n=== Itinerario Lite generado ===")
print(content_lite)

with open("itinerario_lite.txt", "w", encoding="utf-8") as f:
    f.write(content_lite)

# =======================
# Extraer puntos clave del itinerario
# =======================
puntos = []
for line in content_lite.splitlines():
    if any(keyword in line.lower() for keyword in ["actividad", "almuerzo", "cena", "tour", "visita", "excursión", "paseo"]):
        puntos.append(line.strip())
lista_puntos = " | ".join(puntos)

# =======================
# Prompts de Imagen (mapa dinámico + flyer)
# =======================
prompts_img = {
    "Mapa": f"""
Mapa turístico ilustrado de {destino} basado en el itinerario.
Debe incluir un contorno simple del área de viaje y ubicar íconos en los siguientes puntos clave:
{lista_puntos}

Usar solo íconos (vino, montañas, bicicleta, lago, comida, etc.), sin texto ni leyendas.
Fondo con color cálido (arena o terracota), estilo turístico minimalista.
Formato 16:9, imagen clara y coherente.
Devuelve solo la imagen en PNG.
""",

    "Flyer": f"""
Flyer turístico 16:9 para {destino}, estilo travel-poster moderno.
Título: 'Escapada a {destino}'.
Subtítulo: {intereses}.
Imagen icónica del lugar, composición limpia, margen seguro para texto.
Devuelve únicamente la imagen en PNG.
"""
}

# =======================
# Función para generar imágenes
# =======================
def generar_imagenes_lite(prompts: dict, prefix="prompt5lite"):
    for nombre, p in prompts.items():
        try:
            print(f"\n=== Generando {nombre} ===")
            response_img = gemini_client.models.generate_content(
                model="gemini-2.5-flash-image-preview",
                contents=[p],
            )
            saved = False
            for part in response_img.candidates[0].content.parts:
                if hasattr(part, "inline_data") and part.inline_data:
                    img = Image.open(BytesIO(part.inline_data.data))
                    img.save(f"{prefix}_{nombre}.png")
                    print(f"✅ {nombre} generado → {prefix}_{nombre}.png")
                    saved = True
            if not saved:
                print(f"⚠️ No se generó imagen para {nombre}. Gemini no devolvió inline_data")
        except Exception as e:
            print(f"❌ Error generando {nombre}: {e}")

# =======================
# Generar imágenes
# =======================
generar_imagenes_lite(prompts_img, prefix="prompt5lite")

*** Prompt 5-Lite - Intake → Tokens usados: 315 (entrada=195, salida=120), USD 0.000101
=== Prompt 5 Lite — Intake JSON ===
{
  "param": {
    "dest": "Mendoza",
    "dias": 3,
    "pers": 2,
    "presup_pp": {
      "m": 200000,
      "mon": "ARS"
    },
    "int": [
      "Deportes Invernales"
    ],
    "modo": "Exprímelo",
    "fechas": "20/7/2025"
  },
  "sup": [],
  "ajus": [],
  "crit": {},
  "presup": {},
  "chk": [],
  "alert": [],
  "qa": []
}
*** Prompt 5-Lite - Itinerario → Tokens usados: 844 (entrada=254, salida=590), USD 0.000392

=== Itinerario Lite generado ===
**Itinerario de 3 días en Mendoza - Deportes Invernales**

**Día 1 - Valle de Uco**
09:00-11:00 Traslado a Valle de Uco  
11:15-13:00 Esquí en Los Andes  
13:15-14:30 Almuerzo  
- Opción 1: Restaurante "El Refugio"  
- Opción 2: Parador "La Montaña"  
15:00-17:00 Snowboard en la estación de esquí  
17:15-19:00 Clase de esquí para principiantes  
20:00 Cena  
- Opción 1: "La Cabaña de los Esquiadores"  
- Opción 2